In [ ]:
!pip install transformers -U
import torch
from datasets import load_dataset
from transformers import AutoTokenizer

# --- Configuration ---
model_id = "microsoft/Phi-3-mini-4k-instruct"
dataset_id = "b-mc2/sql-create-context"
# This will be the name of your model repository on the Hugging Face Hub
new_model_name = "phi-3-mini-sql-assistant"

# --- Load Dataset ---
# We load the dataset and then, for this educational project, we'll shuffle it
# and select a smaller portion to make training faster.
dataset = load_dataset(dataset_id, split="train")
dataset = dataset.shuffle(seed=42).select(range(10000)) # Using 10k examples for speed

# Split our data into a training set and a small test set to check our work later
dataset = dataset.train_test_split(test_size=0.1)
print(f"Training set size: {len(dataset['train'])}")
print(f"Test set size: {len(dataset['test'])}")

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# A small but important detail: we need to add a padding token.
# We'll set it to the end-of-sequence token.
tokenizer.pad_token = tokenizer.eos_token

# --- Create the Prompt Template ---
def format_prompt(example):
    # This function creates a single string that follows the Phi-3 chat template
    prompt = f"""<|user|>
Given the database schema:
{example['context']}

Generate the SQL query for the following request:
{example['question']}<|end|>
<|assistant|>
{example['answer']}<|end|>"""
    return {"text": prompt}

# Now, apply this formatting to our entire dataset
train_dataset = dataset["train"].map(format_prompt)
test_dataset = dataset["test"].map(format_prompt)

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer

# --- Configuration ---
model_id = "microsoft/Phi-3-mini-4k-instruct"
dataset_id = "b-mc2/sql-create-context"
# This will be the name of your model repository on the Hugging Face Hub
new_model_name = "phi-3-mini-sql-assistant"

# --- Load Dataset ---
# We load the dataset and then, for this educational project, we'll shuffle it
# and select a smaller portion to make training faster.
dataset = load_dataset(dataset_id, split="train")
dataset = dataset.shuffle(seed=42).select(range(10000)) # Using 10k examples for speed

# Split our data into a training set and a small test set to check our work later
dataset = dataset.train_test_split(test_size=0.1)
print(f"Training set size: {len(dataset['train'])}")
print(f"Test set size: {len(dataset['test'])}")

# --- Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# A small but important detail: we need to add a padding token.
# We'll set it to the end-of-sequence token.
tokenizer.pad_token = tokenizer.eos_token

# --- Create the Prompt Template ---
def format_prompt(example):
    # This function creates a single string that follows the Phi-3 chat template
    prompt = f"""<|user|>
Given the database schema:
{example['context']}

Generate the SQL query for the following request:
{example['question']}<|end|>
<|assistant|>
{example['answer']}<|end|>"""
    return {"text": prompt}

# Now, apply this formatting to our entire dataset
train_dataset = dataset["train"].map(format_prompt)
test_dataset = dataset["test"].map(format_prompt)

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import gc # Import garbage collector

# --- Clear GPU memory before loading model ---
torch.cuda.empty_cache()
gc.collect()
print("Attempting to clear GPU memory before model load.")

# --- 1. Configure Quantization (Shrinking the model) ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# --- 2. Load the Base Model ---
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # Removed device_map="auto" as it's causing conflicts with quantization and offloading
    trust_remote_code=True,
)
model.config.use_cache = False # Important for training

# --- 3. Configure LoRA (The Adapters) ---
peft_config = LoraConfig(
    r=8, # Rank of the adapters. A higher rank means more parameters to train.
    lora_alpha=16, # A scaling factor for the adapters.
    lora_dropout=0.1, # Helps prevent overfitting.
    bias="none",
    task_type="CAUSAL_LM",
    # We target these specific modules in the Phi-3 architecture
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# Prepare the model for training with PEFT
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# --- 4. Configure Training Arguments ---
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4, # Simulates a larger batch size (2*4=8)
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=25,
    fp16=True,
    report_to="tensorboard",
)

# --- 5. Create and Run the Trainer ---
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    args=training_args
)

# Let's train!
print("Starting training...")
trainer.train()
print("Training complete!")

# --- 6. Save and Push Your Work ---
print("Saving the adapter to the Hub...")
trainer.model.push_to_hub(new_model_name, use_auth_token=True)
tokenizer.push_to_hub(new_model_name, use_auth_token=True)
print("All done! Your model is saved online.")

# --- Explicitly clear GPU memory after training ---
del model
del trainer
torch.cuda.empty_cache()
gc.collect()
print("GPU memory cleared.")

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import BitsAndBytesConfig # Import BitsAndBytesConfig
import random
import torch # Ensure torch is imported

print("\n--- Testing the fine-tuned model ---")

# --- Re-define Quantization Configuration for Inference ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# --- Load the fine-tuned model ---
# AutoPeftModelForCausalLM automatically loads the base model and applies the adapter.
full_model_name = "manuelaschrittwieser/phi-3-mini-sql-assistant"

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    full_model_name,
    quantization_config=bnb_config, # Pass the quantization config here
    low_cpu_mem_usage=True,
    return_dict=True,
    dtype=torch.float16
)

# --- Merge the adapter into the base model ---
merged_model = trained_model.merge_and_unload()

# --- Test with a random example from the test set ---
sample = dataset["test"][random.randint(0, len(dataset["test"])-1)]
context = sample['context']
question = sample['question']
ground_truth_sql = sample['answer']

# Create the prompt for inference (without the answer!)
prompt = f"""<|user|>
Given the database schema:
{context}

Generate the SQL query for the following request:
{question}<|end|>
<|assistant|>
"""

# Tokenize the prompt and send it to the GPU
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

# Generate the output
outputs = merged_model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=False, use_cache=False)

# Decode only the newly generated tokens (after the input_ids)
generated_output_tokens = outputs[0][len(input_ids[0]):]
generated_sql = tokenizer.decode(generated_output_tokens, skip_special_tokens=True).strip()

# --- Post-process generated SQL to extract only the query ---
# Remove conversational text and markdown code block if present
if "```sql" in generated_sql:
    generated_sql = generated_sql.split("```sql")[1].split("```")[0].strip()

# Remove any lingering <|end|> tokens if present at the end of the generated SQL
if generated_sql.endswith("<|end|>"):
    generated_sql = generated_sql[:-len("<|end|>")].strip()

# --- Print the results ---
print(f"Schema:\n{context}")
print(f"\nQuestion: {question}")
print(f"\n✅ Ground Truth SQL: {ground_truth_sql}")
print(f"🤖 Generated SQL:    {generated_sql}")